In [ ]:
# 🎯 Test Loss: 0.2619 | Test Accuracy: 0.9511
# 🎯 Test Loss: 0.4650 | Test Accuracy: 0.8975
# 🎯 Test Loss: 0.3158 | Test Accuracy: 0.9283
# 病变🎯 Test Loss: 0.6364 | Test Accuracy: 0.8597

In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights
from tqdm.notebook import tqdm

# —— 1. 忽略隐藏目录 —— #
class FilteredImageFolder(ImageFolder):
    def find_classes(self, directory):
        classes = [d.name for d in os.scandir(directory) if d.is_dir() and not d.name.startswith('.')]
        classes.sort()
        class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}
        return classes, class_to_idx

# —— 2. 配置 —— #
train_dir     = "/root/autodl-fs/isic19_20_split/train"
val_dir       = "/root/autodl-fs/isic19_20_split/val"
test_dir      = "/root/autodl-fs/isic19_20_split/test"
ckpt_path     = "/root/autodl-fs/best_effnetv2.pth"
batch_size    = 32
num_epochs    = 10
learning_rate = 1e-4  # EfficientNetV2 建议较小 lr
device        = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# —— 3. 官方预处理 —— #
weights = EfficientNet_V2_S_Weights.DEFAULT
transform = weights.transforms()

# —— 4. 加载数据 —— #
train_dataset = FilteredImageFolder(root=train_dir, transform=transform)
test_dataset  = FilteredImageFolder(root=test_dir,  transform=transform)

if os.path.exists(val_dir):
    val_dataset = FilteredImageFolder(root=val_dir, transform=transform)
else:
    val_len = int(len(train_dataset) * 0.15)
    train_len = len(train_dataset) - val_len
    train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_len, val_len])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=4)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=4)

# —— 5. 模型定义 —— #
model = efficientnet_v2_s(weights=weights)
num_classes = len(train_dataset.dataset.classes if hasattr(train_dataset, 'dataset') else train_dataset.classes)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# —— 6. 训练 + 验证 + 保存最佳模型 —— #
best_val_acc = 0.0
for epoch in range(num_epochs):
    model.train()
    train_loss, train_acc = 0.0, 0
    for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * x.size(0)
        train_acc += (out.argmax(1) == y).sum().item()
    
    train_loss /= len(train_loader.dataset)
    train_acc  /= len(train_loader.dataset)

    # 验证
    model.eval()
    val_loss, val_acc = 0.0, 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            val_loss += loss.item() * x.size(0)
            val_acc  += (out.argmax(1) == y).sum().item()
    val_loss /= len(val_loader.dataset)
    val_acc  /= len(val_loader.dataset)

    print(f"[Epoch {epoch+1}] Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        os.makedirs(os.path.dirname(ckpt_path), exist_ok=True)
        torch.save(model.state_dict(), ckpt_path)
        print("✅ Saved Best Model!")

    # 每个 epoch 保存一次模型
    epoch_ckpt_path = f"/root/autodl-fs/ckpt_effnetv2/epoch_{epoch+1}.pth"
    os.makedirs(os.path.dirname(epoch_ckpt_path), exist_ok=True)
    torch.save(model.state_dict(), epoch_ckpt_path)

# —— 7. 加载并评估测试集 —— #
model.load_state_dict(torch.load(ckpt_path))
model.eval()
test_loss, test_acc, total = 0.0, 0, 0
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        out = model(x)
        loss = criterion(out, y)
        test_loss += loss.item() * x.size(0)
        test_acc  += (out.argmax(1) == y).sum().item()
        total += y.size(0)

test_loss /= total
test_acc  /= total
print(f"🎯 Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")

# —— 8. 所有 epoch 的模型推理 —— #
print("\n📊 Evaluating all epoch checkpoints on test set:")
epoch_results = []

for e in range(1, num_epochs + 1):
    ckpt_file = f"/root/autodl-fs/ckpt_effnetv2/epoch_{e}.pth"
    if not os.path.exists(ckpt_file):
        print(f"❌ Epoch {e} model not found.")
        continue

    model.load_state_dict(torch.load(ckpt_file))
    model.eval()

    test_loss, test_acc, total = 0.0, 0, 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            test_loss += loss.item() * x.size(0)
            test_acc  += (out.argmax(1) == y).sum().item()
            total += y.size(0)

    test_loss /= total
    test_acc  /= total
    epoch_results.append((e, test_loss, test_acc))
    print(f"📁 Epoch {e:02d} | Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")


Epoch 1/10:   0%|          | 0/251 [00:00<?, ?it/s]

[Epoch 1] Train Loss: 0.2469, Acc: 0.9062 | Val Loss: 0.1704, Acc: 0.9377
✅ Saved Best Model!


Epoch 2/10:   0%|          | 0/251 [00:00<?, ?it/s]

[Epoch 2] Train Loss: 0.1480, Acc: 0.9431 | Val Loss: 0.1582, Acc: 0.9435
✅ Saved Best Model!


Epoch 3/10:   0%|          | 0/251 [00:00<?, ?it/s]

...

[Epoch 10] Train Loss: 0.0206, Acc: 0.9933 | Val Loss: 0.2505, Acc: 0.9394


/tmp/ipykernel_48658/1118557471.py:102: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ckpt_path))


🎯 Test Loss: 0.2619 | Test Accuracy: 0.9511

📊 Evaluating all epoch checkpoints on test set:


/tmp/ipykernel_48658/1118557471.py:128: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ckpt_file))


📁 Epoch 01 | Test Loss: 0.1666 | Test Accuracy: 0.9325
📁 Epoch 02 | Test Loss: 0.1485 | Test Accuracy: 0.9383
📁 Epoch 03 | Test Loss: 0.1701 | Test Accuracy: 0.9383
📁 Epoch 04 | Test Loss: 0.1780 | Test Accuracy: 0.9476
📁 Epoch 05 | Test Loss: 0.1678 | Test Accuracy: 0.9523
📁 Epoch 06 | Test Loss: 0.2350 | Test Accuracy: 0.9470
📁 Epoch 07 | Test Loss: 0.2353 | Test Accuracy: 0.9360
📁 Epoch 08 | Test Loss: 0.2943 | Test Accuracy: 0.9249
📁 Epoch 09 | Test Loss: 0.2619 | Test Accuracy: 0.9511
📁 Epoch 10 | Test Loss: 0.2479 | Test Accuracy: 0.9459


In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights
from tqdm.notebook import tqdm

# —— 1. 忽略隐藏目录 —— #
class FilteredImageFolder(ImageFolder):
    def find_classes(self, directory):
        classes = [d.name for d in os.scandir(directory) if d.is_dir() and not d.name.startswith('.')]
        classes.sort()
        class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}
        return classes, class_to_idx

# —— 2. 配置 —— #
train_dir     = "/root/autodl-fs/processed/processed/train"
val_dir       = "/root/autodl-fs/processed/processed/val"
test_dir      = "/root/autodl-fs/processed/processed/test"
ckpt_path     = "/root/autodl-fs/best_effnetv2.pth"
batch_size    = 32
num_epochs    = 10
learning_rate = 1e-4  # EfficientNetV2 建议较小 lr
device        = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# —— 3. 官方预处理 —— #
weights = EfficientNet_V2_S_Weights.DEFAULT
transform = weights.transforms()

# —— 4. 加载数据 —— #
train_dataset = FilteredImageFolder(root=train_dir, transform=transform)
test_dataset  = FilteredImageFolder(root=test_dir,  transform=transform)

if os.path.exists(val_dir):
    val_dataset = FilteredImageFolder(root=val_dir, transform=transform)
else:
    val_len = int(len(train_dataset) * 0.15)
    train_len = len(train_dataset) - val_len
    train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_len, val_len])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=4)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=4)

# —— 5. 模型定义 —— #
model = efficientnet_v2_s(weights=weights)
num_classes = len(train_dataset.dataset.classes if hasattr(train_dataset, 'dataset') else train_dataset.classes)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# —— 6. 训练 + 验证 + 保存最佳模型 —— #
best_val_acc = 0.0
for epoch in range(num_epochs):
    model.train()
    train_loss, train_acc = 0.0, 0
    for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * x.size(0)
        train_acc += (out.argmax(1) == y).sum().item()
    
    train_loss /= len(train_loader.dataset)
    train_acc  /= len(train_loader.dataset)

    # 验证
    model.eval()
    val_loss, val_acc = 0.0, 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            val_loss += loss.item() * x.size(0)
            val_acc  += (out.argmax(1) == y).sum().item()
    val_loss /= len(val_loader.dataset)
    val_acc  /= len(val_loader.dataset)

    print(f"[Epoch {epoch+1}] Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        os.makedirs(os.path.dirname(ckpt_path), exist_ok=True)
        torch.save(model.state_dict(), ckpt_path)
        print("✅ Saved Best Model!")

    # 每个 epoch 保存一次模型
    epoch_ckpt_path = f"/root/autodl-fs/ckpt_effnetv2/epoch_{epoch+1}.pth"
    os.makedirs(os.path.dirname(epoch_ckpt_path), exist_ok=True)
    torch.save(model.state_dict(), epoch_ckpt_path)

# —— 7. 加载并评估测试集 —— #
model.load_state_dict(torch.load(ckpt_path))
model.eval()
test_loss, test_acc, total = 0.0, 0, 0
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        out = model(x)
        loss = criterion(out, y)
        test_loss += loss.item() * x.size(0)
        test_acc  += (out.argmax(1) == y).sum().item()
        total += y.size(0)

test_loss /= total
test_acc  /= total
print(f"🎯 Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")

# —— 8. 所有 epoch 的模型推理 —— #
print("\n📊 Evaluating all epoch checkpoints on test set:")
epoch_results = []

for e in range(1, num_epochs + 1):
    ckpt_file = f"/root/autodl-fs/ckpt_effnetv2/epoch_{e}.pth"
    if not os.path.exists(ckpt_file):
        print(f"❌ Epoch {e} model not found.")
        continue

    model.load_state_dict(torch.load(ckpt_file))
    model.eval()

    test_loss, test_acc, total = 0.0, 0, 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            test_loss += loss.item() * x.size(0)
            test_acc  += (out.argmax(1) == y).sum().item()
            total += y.size(0)

    test_loss /= total
    test_acc  /= total
    epoch_results.append((e, test_loss, test_acc))
    print(f"📁 Epoch {e:02d} | Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")


Epoch 1/10:   0%|          | 0/287 [00:00<?, ?it/s]

[Epoch 1] Train Loss: 0.2416, Acc: 0.9081 | Val Loss: 0.1703, Acc: 0.9350
✅ Saved Best Model!


Epoch 2/10:   0%|          | 0/287 [00:00<?, ?it/s]

[Epoch 2] Train Loss: 0.1312, Acc: 0.9507 | Val Loss: 0.1719, Acc: 0.9424
✅ Saved Best Model!


Epoch 3/10:   0%|          | 0/287 [00:00<?, ?it/s]

...

[Epoch 10] Train Loss: 0.0189, Acc: 0.9939 | Val Loss: 0.3878, Acc: 0.9449
✅ Saved Best Model!


/tmp/ipykernel_1284/1116636306.py:102: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ckpt_path))


🎯 Test Loss: 0.5009 | Test Accuracy: 0.9092

📊 Evaluating all epoch checkpoints on test set:


/tmp/ipykernel_1284/1116636306.py:128: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ckpt_file))


📁 Epoch 01 | Test Loss: 0.3043 | Test Accuracy: 0.8960
📁 Epoch 02 | Test Loss: 0.2265 | Test Accuracy: 0.9195
📁 Epoch 03 | Test Loss: 0.4739 | Test Accuracy: 0.8858
📁 Epoch 04 | Test Loss: 0.4795 | Test Accuracy: 0.9019
📁 Epoch 05 | Test Loss: 0.4650 | Test Accuracy: 0.8975
📁 Epoch 06 | Test Loss: 0.4047 | Test Accuracy: 0.9092
📁 Epoch 07 | Test Loss: 0.5012 | Test Accuracy: 0.9165
📁 Epoch 08 | Test Loss: 0.4475 | Test Accuracy: 0.9180
📁 Epoch 09 | Test Loss: 0.5752 | Test Accuracy: 0.9019
📁 Epoch 10 | Test Loss: 0.5009 | Test Accuracy: 0.9092


In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights
from tqdm.notebook import tqdm

# —— 1. 忽略隐藏目录 —— #
class FilteredImageFolder(ImageFolder):
    def find_classes(self, directory):
        classes = [d.name for d in os.scandir(directory) if d.is_dir() and not d.name.startswith('.')]
        classes.sort()
        class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}
        return classes, class_to_idx

# —— 2. 配置 —— #
train_dir     = "/root/autodl-fs/generate/train"
val_dir       = "/root/autodl-fs/generate/val"
test_dir      = "/root/autodl-fs/generate/test"
ckpt_path     = "/root/autodl-fs/best_effnetv2.pth"
batch_size    = 32
num_epochs    = 10
learning_rate = 1e-4  # EfficientNetV2 建议较小 lr
device        = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# —— 3. 官方预处理 —— #
weights = EfficientNet_V2_S_Weights.DEFAULT
transform = weights.transforms()

# —— 4. 加载数据 —— #
train_dataset = FilteredImageFolder(root=train_dir, transform=transform)
test_dataset  = FilteredImageFolder(root=test_dir,  transform=transform)

if os.path.exists(val_dir):
    val_dataset = FilteredImageFolder(root=val_dir, transform=transform)
else:
    val_len = int(len(train_dataset) * 0.15)
    train_len = len(train_dataset) - val_len
    train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_len, val_len])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=4)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=4)

# —— 5. 模型定义 —— #
model = efficientnet_v2_s(weights=weights)
num_classes = len(train_dataset.dataset.classes if hasattr(train_dataset, 'dataset') else train_dataset.classes)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# —— 6. 训练 + 验证 + 保存最佳模型 —— #
best_val_acc = 0.0
for epoch in range(num_epochs):
    model.train()
    train_loss, train_acc = 0.0, 0
    for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * x.size(0)
        train_acc += (out.argmax(1) == y).sum().item()
    
    train_loss /= len(train_loader.dataset)
    train_acc  /= len(train_loader.dataset)

    # 验证
    model.eval()
    val_loss, val_acc = 0.0, 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            val_loss += loss.item() * x.size(0)
            val_acc  += (out.argmax(1) == y).sum().item()
    val_loss /= len(val_loader.dataset)
    val_acc  /= len(val_loader.dataset)

    print(f"[Epoch {epoch+1}] Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        os.makedirs(os.path.dirname(ckpt_path), exist_ok=True)
        torch.save(model.state_dict(), ckpt_path)
        print("✅ Saved Best Model!")

    # 每个 epoch 保存一次模型
    epoch_ckpt_path = f"/root/autodl-fs/ckpt_effnetv2/epoch_{epoch+1}.pth"
    os.makedirs(os.path.dirname(epoch_ckpt_path), exist_ok=True)
    torch.save(model.state_dict(), epoch_ckpt_path)

# —— 7. 加载并评估测试集 —— #
model.load_state_dict(torch.load(ckpt_path))
model.eval()
test_loss, test_acc, total = 0.0, 0, 0
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        out = model(x)
        loss = criterion(out, y)
        test_loss += loss.item() * x.size(0)
        test_acc  += (out.argmax(1) == y).sum().item()
        total += y.size(0)

test_loss /= total
test_acc  /= total
print(f"🎯 Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")

# —— 8. 所有 epoch 的模型推理 —— #
print("\n📊 Evaluating all epoch checkpoints on test set:")
epoch_results = []

for e in range(1, num_epochs + 1):
    ckpt_file = f"/root/autodl-fs/ckpt_effnetv2/epoch_{e}.pth"
    if not os.path.exists(ckpt_file):
        print(f"❌ Epoch {e} model not found.")
        continue

    model.load_state_dict(torch.load(ckpt_file))
    model.eval()

    test_loss, test_acc, total = 0.0, 0, 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            test_loss += loss.item() * x.size(0)
            test_acc  += (out.argmax(1) == y).sum().item()
            total += y.size(0)

    test_loss /= total
    test_acc  /= total
    epoch_results.append((e, test_loss, test_acc))
    print(f"📁 Epoch {e:02d} | Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")


Epoch 1/10:   0%|          | 0/283 [00:00<?, ?it/s]

[Epoch 1] Train Loss: 0.3344, Acc: 0.8547 | Val Loss: 0.2477, Acc: 0.8966
✅ Saved Best Model!


Epoch 2/10:   0%|          | 0/283 [00:00<?, ?it/s]

[Epoch 2] Train Loss: 0.2293, Acc: 0.9101 | Val Loss: 0.2419, Acc: 0.9010
✅ Saved Best Model!


Epoch 3/10:   0%|          | 0/283 [00:00<?, ?it/s]

...

[Epoch 10] Train Loss: 0.0291, Acc: 0.9901 | Val Loss: 0.5279, Acc: 0.9029


/tmp/ipykernel_1284/483776496.py:102: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ckpt_path))


🎯 Test Loss: 0.6364 | Test Accuracy: 0.8597

📊 Evaluating all epoch checkpoints on test set:


/tmp/ipykernel_1284/483776496.py:128: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ckpt_file))


📁 Epoch 01 | Test Loss: 0.2826 | Test Accuracy: 0.8804
📁 Epoch 02 | Test Loss: 0.2772 | Test Accuracy: 0.8966
📁 Epoch 03 | Test Loss: 0.3455 | Test Accuracy: 0.8877
📁 Epoch 04 | Test Loss: 0.3910 | Test Accuracy: 0.8863
📁 Epoch 05 | Test Loss: 0.4106 | Test Accuracy: 0.8700
📁 Epoch 06 | Test Loss: 0.5313 | Test Accuracy: 0.8700
📁 Epoch 07 | Test Loss: 0.4930 | Test Accuracy: 0.8612
📁 Epoch 08 | Test Loss: 0.4338 | Test Accuracy: 0.8789
📁 Epoch 09 | Test Loss: 0.6364 | Test Accuracy: 0.8597
📁 Epoch 10 | Test Loss: 0.8108 | Test Accuracy: 0.8877
